In [ ]:
import json

def create_initial_state(state_file="/kaggle/working/state.json"):
    """
    Creates a starting state file for the member's 8-month journey.
    Includes placeholders for test_panel, wearable data, and member questions.
    """
    state = {
        "patient_info": {
            "name": "Rohan",
            "age": 45,
            "location": "Singapore",
            "chronic_condition": "high blood sugar",
            "living_situation": "lives alone",
            "job": "high-level executive, frequent travel"
        },
        "health_state": {
            "major_event": None,
            "minor_event": None,
            "current_symptoms": [],
            "adherence_level": 1.0,
            "next_test_due": "Week 12"
        },
        "plan": {
            "exercise": "light walking",
            "diet": "low sugar",
            "medication_reminders": True,
            "supplements": []
        },
        "wearable": {
            "week_0": None,     # To be filled by wearable prompt output
            "biweekly_updates": {}  # e.g. {"week_2": {...}, "week_4": {...}}
        },
        "member_questions": {
            "weekly": {}        # e.g. {"week_1": [...], "week_2": [...]}
        },
        "last_week_summary": {
            "topics_discussed": [],
            "elyx_team_involved": [],
            "key_decisions": []
        },
        "logistics": {
            "travel_schedule": [],
            "preferred_contact_times": [],
            "assistant_contact": None
        }
    }

    with open(state_file, "w") as f:
        json.dump(state, f, indent=2)
    print(f"Initial state file created: {state_file}")


# --- Example usage ---
create_initial_state()  # This will create state.json in your current directory


In [ ]:
import json

def load_state(state_file="state.json"):
    """Load the state file into a Python dictionary."""
    with open(state_file, "r") as f:
        state = json.load(f)
    return state

# Example usage
state = load_state()

In [ ]:

test_panel_prompt = f"""
You are generating a comprehensive health report based on the following test panel. 
This panel is always performed at the start of the program (Week 0) and again at Week 12 
to evaluate the member's progress.

IMPORTANT INSTRUCTIONS:
- Keep the test panel content EXACTLY as given below.
- Use it to create a structured, realistic report.
- For Week 0: Generate baseline findings for each category. 
- For Week 12: Show realistic progress — most parameters should improve if the plan is followed, 
  but some may stay unchanged or even worsen due to side effects, lifestyle, or partial adherence.
- Always specify which parameters are OUT OF RANGE, which have IMPROVED, which remain UNCHANGED, 
  and which have WORSENED.
- Make the report sound like it is written by medical professionals, using clear, non-technical 
  language where appropriate.
- Mention any recommendations or action items at the end.
- DO NOT add or remove tests from the panel.

TEST PANEL (DO NOT MODIFY):

1. General Health Assessment:
- Clinical History: A thorough interview to study the client’s current health.
- Physical Examination: Comprehensive assessment by a physician.
- Vital Signs: Blood pressure, heart rate, and anthropometry like body mass index (BMI).
- Blood Tests:
  OGTT with paired insulin
  Lipid profile (cholesterol levels) + advanced lipid tests e.g. ApoB/ApoA, Lp(a), PLAC test, etc
  Full blood count (FBC)
  Liver and kidney function tests
  Micronutrient Panel - including Omega-3
  ESR CRP
  Biological: TruAge
  TSH T3 T4, Cortisol
  Sex Hormones: Age-adjusted
  Heavy Metals: Lead, Mercury
  ApoE4
  Epigenetic tests
  Urinalysis: Kidney and urinary tract health.

2. Cancer Screening:
- Colorectal Cancer:
  Faecal Immunochemical Test (FIT): Detects blood in stools (annually).
  Colonoscopy: More comprehensive screening (every 5-10 years).
  KIV Full Body MRI OR Targeted MRI /Lucence
- Cervical Cancer (Women):
  Cervical smear (25-29 years old): Every 3 years.
  HPV Test (30 years old and above): Every 5 years.
- Breast Cancer (Women):
  Mammogram: Biennial screening (50-69 years old).

3. Advanced Cardiovascular Assessment:
- Electrocardiogram (ECG): Detects cardiac rhythm abnormalities.
- Coronary Calcium Score with Angiography: Detects early signs of heart disease. Vs Cleerly
- Echocardiogram: KIV if no full body MRI Detailed heart imaging.
- Carotid Intima-Media Thickness (CIMT) Scan: Assess carotid artery health.

4. Overall health and fitness:
- VO2 Max Testing: Provides insights into aerobic capacity and endurance [adjusted to conditioning level]/prior PARQ assessment]
- Grip strength assessment: Well correlated with longevity and overall health
- Functional Movement Screening (FMS): Assesses movement patterns, balance, and stability
- Indirect Calorimetry/DLW
- Spirometry

5. Genetic Testing:
- Hereditary Risk Assessment: Based on family history.
- Pharmacogenomics: Personalised drug response prediction.

6. Body Composition Analysis:
- DEXA Scan: Measures bone density, fat, and muscle mass.

7. Hormone Profiling:
- Thyroid Function Tests: TSH, T3, T4 levels.
- Sex Hormone Levels: Estradiol, testosterone, progesterone.

8. Nutritional Assessment:
- Micronutrient Levels: Vitamins, minerals, antioxidants.
- Food Allergy Testing: Identify sensitivities.
- Gut microbiome

9. Brain Health Assessment:
- Cognitive Function Tests: Memory, attention, executive function.
- Mental health review: Assess mood, stress
- Brain MRI: MRI stroke screen Detects early signs of neurodegenerative diseases.

10. Skin Analysis
- VISIA

11. Extended care:
- Comprehensive Consultation: With specialists (cardiologist, endocrinologist, etc.).
- Personalised Lifestyle Recommendations: Nutrition, exercise, stress management.

TASK:
- First, generate a Week 0 baseline report with realistic findings.
- Then, generate a Week 12 follow-up report showing changes.
- Explicitly point out:
  * Improvements
  * Parameters still out of range
  * Any new issues or side effects
  * Recommendations for the next stage

Format clearly and professionally.
"""
import json
import re
from openai import OpenAI

client = OpenAI(api_key=api_key)  # assumes api_key already loaded

def generate_test_panel_report(client, test_panel_prompt):
    """
    Generates a simplified test panel report using GPT.
    Picks a few key parameters (not exhaustive) and assigns realistic numbers.
    Cleans illegal control characters to avoid JSONDecodeError.
    """
    prompt = f"""
You are Elyx, a personalized health service.
The following is the test panel performed at week 0 and week 12:
{test_panel_prompt}

INSTRUCTIONS:
- Do NOT create a very detailed or exhaustive report.
- Choose only a few representative parameters (e.g., cholesterol, blood pressure, glucose, VO2 max, liver enzymes).
- Assign realistic numerical values and comment if they are good or bad.
- For Week 0: show baseline numbers, some out of range.
- For Week 12: some improvements, some unchanged, one or two worsened.
- For Week 24: mostly stable, in maintenance phase.
- Output ONLY valid JSON in this format:
{{
  "summary": "2-3 sentence overview of current health",
  "detailed_report": [
    "Parameter X: value (status/change)",
    "Parameter Y: value (status/change)",
    "Parameter Z: value (status/change)"
  ]
}}
DO NOT add any extra explanation or labels like 'Here is the JSON'.
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",  # switch to gpt-3.5-turbo if you want
        messages=[
            {"role": "system", "content": "You are a helpful assistant that writes structured health reports."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=800,
        temperature=0.7
    )

    raw_output = response.choices[0].message.content.strip()

    # --- Extract JSON ---
    json_match = re.search(r"\{.*\}", raw_output, re.S)
    if not json_match:
        raise ValueError(f"No valid JSON found in model output:\n{raw_output}")

    json_str = json_match.group(0)

    # --- Clean illegal characters ---
    json_str = re.sub(r'[\x00-\x1F\x7F]', ' ', json_str)  # remove control chars

    try:
        report = json.loads(json_str)
    except json.JSONDecodeError as e:
        raise ValueError(f"Failed to parse JSON after cleaning: {e}\nContent:\n{json_str}")

    return report


In [ ]:
import json

def generate_wearable_update(client, week_num, conversation_text=None):
    """
    Generates a structured wearable-device status update for the given week.
    Returns a Python dict that can be directly saved into state.json.
    """
    wearable_prompt = f"""
Assume you are Elyx, monitoring a member's health using wearable device data.

TASK:
- In Week {week_num}, decide if the member:
  * is using a wearable device, or
  * should be recommended to start, continue, switch, or stop using one.
- Assume Elyx consistently receives and reviews wearable data.
- Generate 1–2 realistic issues in the data:
  * Positive (e.g., improved sleep quality, better activity consistency)
  * Negative (e.g., irregular HR readings, sync gaps, device discomfort)
- If the member expresses dissatisfaction (comfort, accuracy, battery life), describe it.
- Recommend Elyx's response: troubleshoot, reassure, escalate to a doctor, or replace/switch device.
- Make recommendations detailed, practical, and consistent with real-world wearable tracking.

OUTPUT REQUIREMENTS:
Respond ONLY in valid JSON with this structure:
{{
  "week": {week_num},
  "wearable_in_use": "Yes/No",
  "device_name": "Fitbit / Apple Watch / Garmin / etc",
  "detected_issues": ["issue1", "issue2"],
  "member_feedback": "Positive / Negative / Neutral",
  "elyx_recommendation": "Continue device / Change device / Stop using device / Start using device",
  "next_steps": ["step1", "step2"]
}}

Conversation context (if available):
{conversation_text if conversation_text else "No additional conversation context"}
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a medical assistant who outputs structured JSON only."},
            {"role": "user", "content": wearable_prompt}
        ],
        max_tokens=400,
        temperature=0.7
    )

    raw_output = response.choices[0].message.content

    # Parse JSON safely
    try:
        wearable_data = json.loads(raw_output)
    except json.JSONDecodeError:
        print("Warning: Model returned invalid JSON. Raw output:")
        print(raw_output)
        wearable_data = {
            "week": week_num,
            "wearable_in_use": "Unknown",
            "device_name": "Unknown",
            "detected_issues": [],
            "member_feedback": "Neutral",
            "elyx_recommendation": "No recommendation",
            "next_steps": []
        }

    return wearable_data


In [ ]:
import json

def generate_member_questions(client, week_num, conversation_text=None):
    """
    Generates realistic health-related questions from the member for a given week.
    Returns a Python dict that can be directly saved into state.json.
    """
    member_prompt = f"""
Assume the member (patient) is proactive, does independent research, and prefers evidence-based approaches. 
Each week, he initiates at least one conversation with Elyx by asking thoughtful, specific questions.

Your task: Generate 2 realistic, varied questions** for Week {week_num}.  
The questions should:
- Relate to his chronic condition, ongoing health journey, or wellness goals.  
- Reference medications, therapies, exercises, diets, wearable data, or recent life events (travel, stress, test results, symptoms).  
- Lead to meaningful, productive discussions with Elyx.  
- Sometimes challenge or seek clarification of Elyx’s recommendations.  
- Sound natural, intelligent, and detailed — not generic.  
- Do not provide answers, only the member's questions.

Respond ONLY in valid JSON with this format:
{{
  "week": {week_num},
  "questions": [
    "question1",
    "question2",
  ]
}}

Conversation context (if available):
{conversation_text if conversation_text else "No additional conversation context"}
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a medical assistant who outputs structured JSON only."},
            {"role": "user", "content": member_prompt}
        ],
        max_tokens=300,
        temperature=0.7
    )

    raw_output = response.choices[0].message.content

    # Parse JSON safely
    try:
        member_data = json.loads(raw_output)
    except json.JSONDecodeError:
        print("Warning: Model returned invalid JSON. Raw output:")
        print(raw_output)
        member_data = {
            "week": week_num,
            "questions": []
        }

    return member_data


In [ ]:
import json

def generate_health_events(client):
    """
    Generates a 4-week health journey for Rohan with:
    - One major issue in Week 1
    - Continuation impact in Weeks 2 and 3
    - One minor issue starting Week 3
    - Recovery or stabilization in Week 4

    Returns a Python dict that can be saved directly into state.json.
    """
    event_prompt = """
You are creating a health journey for Rohan (45, high-level executive, lives alone) 
over 4 weeks. He is generally healthy but has one chronic issue (you choose), 
and unexpected things can happen.

Requirements:
- Invent ONE major issue in Week 1 (e.g. injury, infection, sudden test result).
- Show how this major issue continues to affect Weeks 2 and 3 realistically.
- Invent ONE minor issue starting around Week 3.
- Summarize Week 4 recovery or stabilization.

Respond ONLY with valid JSON:
{
  "week_1_event": "...",
  "week_2_update": "...",
  "week_3_update": "...",
  "week_4_update": "..."
}
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a medical assistant who outputs structured JSON only."},
            {"role": "user", "content": event_prompt}
        ],
        max_tokens=300,
        temperature=0.8
    )

    raw_output = response.choices[0].message.content

    # Parse JSON safely
    try:
        events = json.loads(raw_output)
    except json.JSONDecodeError:
        print("Warning: Model returned invalid JSON. Raw output:")
        print(raw_output)
        events = {
            "week_1_event": "",
            "week_2_update": "",
            "week_3_update": "",
            "week_4_update": ""
        }

    return events


In [ ]:
import json

def update_logistics(state, travel_schedule=None, preferred_contact_times=None, assistant_contact=None, state_file="state.json"):
    """
    Updates the logistics section of the state.json file.
    
    - travel_schedule: list of trips (e.g. [{"week": 2, "location": "Tokyo"}, ...])
    - preferred_contact_times: list of preferred times (e.g. ["morning", "evening"])
    - assistant_contact: dictionary or string with assistant info (e.g. {"name": "Arjun", "phone": "+65..."} or just a name)
    """
    if "logistics" not in state:
        state["logistics"] = {
            "travel_schedule": [],
            "preferred_contact_times": [],
            "assistant_contact": None
        }

    if travel_schedule is not None:
        state["logistics"]["travel_schedule"] = travel_schedule
    if preferred_contact_times is not None:
        state["logistics"]["preferred_contact_times"] = preferred_contact_times
    if assistant_contact is not None:
        state["logistics"]["assistant_contact"] = assistant_contact

    # Save back to file
    with open(state_file, "w") as f:
        json.dump(state, f, indent=2)
    print("Logistics section updated successfully.")


In [ ]:

update_logistics(
    state,
    travel_schedule=[{"week": 2, "location": "Tokyo"}, {"week": 4, "location": "Jakarta"}],
    preferred_contact_times=["morning", "late evening"],
    assistant_contact={"name": "Meera", "phone": "+65-1234-5678"}
)


In [ ]:
import json

def load_state(state_file="state.json"):
    """Load the state JSON file."""
    with open(state_file, "r") as f:
        return json.load(f)

def format_state_as_context(state):
    """Turn the full state dictionary into a compact but detailed context string."""

    # --- Safe accessors ---
    patient = state.get("patient_info", {})
    health = state.get("health_events", {})
    plan = state.get("plan", {})
    logistics = state.get("logistics", {})
    wearables = state.get("last_wearable_update", {})
    member_qs = state.get("last_member_questions", {})
    test_panel = state.get("test_panel", {})
    weekly_summaries = state.get("weekly_summaries", {})

    # --- Helper formatting ---
    def fmt_list(val):
        return ', '.join(val) if isinstance(val, list) else (str(val) if val else 'None')

    def fmt_dict(d, max_items=3):
        """Compactly show key:value pairs, capped to max_items."""
        if not d:
            return "None"
        items = [f"{k}: {str(v)[:50]}{'...' if len(str(v))>50 else ''}" for k,v in list(d.items())[:max_items]]
        more = f" (+{len(d)-max_items} more)" if len(d) > max_items else ""
        return "; ".join(items) + more

    context = f"""
Patient: {patient.get('name', 'Unknown')}, {patient.get('age', '?')} y/o, {patient.get('job', 'Unknown')}, lives in {patient.get('location', 'Unknown')}, chronic condition: {patient.get('chronic_condition', 'Unknown')}.
Current health: Major event: {health.get('major_event', 'None')}, Minor event: {health.get('minor_event', 'None')}, Symptoms: {fmt_list(health.get('current_symptoms', []))}, Adherence: {health.get('adherence_level', 'Unknown')}.
Current plan: Exercise: {plan.get('exercise','None')}, Diet: {plan.get('diet','None')}, Medication reminders: {plan.get('medication_reminders','None')}, Supplements: {fmt_list(plan.get('supplements', []))}.
Weekly summaries: {fmt_dict(weekly_summaries)}
Wearable updates: {fmt_dict(wearables)}
Member questions: {fmt_dict(member_qs)}
Test panel: {fmt_dict(test_panel)}
Logistics: Travel: {logistics.get('travel_schedule', 'None')}, Preferred contact: {logistics.get('preferred_contact_times', 'None')}, Assistant: {logistics.get('assistant_contact', 'None')}.
"""
    return context.strip()



In [ ]:
def generate_week_conversation(week, state, events_summary):
    context = format_state_as_context(state)
    
    prompt = f"""
You are writing WhatsApp-style messages between Rohan and the Elyx team.

Context from state:
{context}

Event history so far:
{events_summary}

Now write Week {week} messages:
### Requirements:
1. **Conversation structure:**  
   - At least **3 distinct conversation threads** within the week.  
   - Messages must feel natural, with realistic timestamps implied by the flow (but **no explicit timestamps in output**).  

2. **Content coverage:**  
   - Include topics about **travel, diet, exercise, symptoms, medication reminders, and coordination**.  
   - Rohan must mention **any major or minor health events** from the context.  
   - Either Rohan or Elyx must mention **updates from wearables** (from context).  
   - Rohan must ask **at least 2 of the member questions** provided in the context.  
   - Elyx team must show **proactive support** (offering suggestions, reminders, follow-ups).  
   - As weeks progress, messages must **show improved diet, exercise, and medication adherence** and **Rohan’s symptoms improving**.
   - On weeks 1,12 and 24 when you get test panel results, talk in detail about the report with rohan.
   
3. **Output format:**  
   - Output **ONLY a valid JSON list** of exactly 20 strings.  
   - **Do not include timestamps, speaker labels, or extra commentary outside messages.**  
   - Each message must begin with the speaker’s name (`"Rohan:"`, `"Doctor:"`, `"Nutritionist:"`, `"Lifestyle Coach:"`).  
   - **No trailing commas, no explanations outside the list.**
[
  {{"speaker": "Ruby (Elyx Concierge)", "message": "..."}},
  {{"speaker": "Rohan", "message": "..."}}
]
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You write detailed, naturalistic health coaching chats."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=1800,
        temperature=0.7
    )
    return response.choices[0].message.content.strip()


In [ ]:
import re
import json

def summarize_conversation(conversation_text):
    prompt = f"""
Summarize the following conversation in JSON with fields:
topics_discussed, elyx_team_involved, key_decisions, major_event, minor_event, current_symptoms, adherence_level (0 to 1).
Conversation:
{conversation_text}
Return ONLY valid JSON (object or list), no extra text.
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You extract structured summaries from chat transcripts. Return only JSON."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=600,
        temperature=0
    )
    
    summary_str = response.choices[0].message.content.strip()

    # Extract JSON from any extra text
    match = re.search(r'(\{.*\}|\[.*\])', summary_str, re.DOTALL)
    if match:
        summary_json_str = match.group(0)
        try:
            summary = json.loads(summary_json_str)
        except json.JSONDecodeError:
            print("Failed to parse JSON after extraction:")
            print(summary_json_str)
            summary = {}
    else:
        print("No JSON object found in model output:")
        print(summary_str)
        summary = {}
    
    return summary


In [ ]:
import json
import os

state_file_path = "/kaggle/working/state.json"

def update_state(state_file_path, state, week, conversation_summary):
    """
    Overwrite the last week's summary in the state dictionary and save to JSON.
    Keeps only the latest week's summary to control prompt length.

    Args:
        state_file_path (str): Path to the state.json file.
        state (dict): Current state dictionary.
        week (int): Week number to update.
        conversation_summary (dict): Structured summary from summarize_conversation().
    """
    if 'weekly_summaries' not in state: 
        state['weekly_summaries'] = {}
    # Always store only the last week summary
    state["weekly_summaries"] = {
        "week": week,
        "topics_discussed": conversation_summary.get("topics_discussed", []),
        "elyx_team_involved": conversation_summary.get("elyx_team_involved", []),
        "key_decisions": conversation_summary.get("key_decisions", []),
        "major_event": conversation_summary.get("major_event"),
        "minor_event": conversation_summary.get("minor_event"),
        "current_symptoms": conversation_summary.get("current_symptoms", []),
        "adherence_level": conversation_summary.get("adherence_level", 1.0)
    }

    # Save back to JSON
    with open(state_file_path, "w", encoding="utf-8") as f:
        json.dump(state, f, indent=4, ensure_ascii=False)

    print(f"✅ Overwrote last_week_summary with Week {week} data in {os.path.basename(state_file_path)}")


In [ ]:
import json
import os
import datetime

state_file_path = "/kaggle/working/state.json"
conversations_file_path = "/kaggle/working/all_conversations.json"
state_log_path = "/kaggle/working/state_changes_log.jsonl"

# --- Helper functions ---
def log_state_change(state, description=""):
    log_entry = {
        "timestamp": datetime.datetime.utcnow().isoformat(),
        "description": description,
        "state": state
    }
    with open(state_log_path, "a", encoding="utf-8") as f:
        f.write(json.dumps(log_entry, ensure_ascii=False) + "\n")

# --- STEP 0: Load or initialize state ---
state = load_state()

if "test_panel" not in state:
    test_report = generate_test_panel_report(client, test_panel_prompt)
    state["test_panel"] = test_report
    log_state_change(state, "Added test_panel report")

if "last_wearable_update" not in state:
    state["last_wearable_update"] = {}
    log_state_change(state, "Initialized last_wearable_update")

if "last_member_questions" not in state:
    state["last_member_questions"] = {}
    log_state_change(state, "Initialized last_member_questions")

with open(state_file_path, "w") as f:
    json.dump(state, f, indent=2)

# --- STEP 1: Load existing conversations ---
if os.path.exists(conversations_file_path):
    with open(conversations_file_path, "r") as f:
        all_conversations = json.load(f)
else:
    all_conversations = {}

#A phase is 8 weeks or 2 months
for phase in range(1, 5):
    print(f"\n=== STARTING PHASE {phase} (Weeks {(phase-1)*8+1} to {phase*8}) ===\n")
    history = ""
    weekly_conversations = {}

    # Generate new health events for this phase
    phase_events_key = f"phase_{phase}_events"
    if phase_events_key not in state:
        state[phase_events_key] = generate_health_events(client)
        log_state_change(state, f"Added {phase_events_key}")

    events = state[phase_events_key]

    for week_in_phase in range(1, 9):
        week_global = (phase - 1) * 8 + week_in_phase

        # Add health events context only for first 4 weeks of each phase
        if week_in_phase <= 4:
            week_key = f"week_{week_in_phase}_event" if week_in_phase == 1 else f"week_{week_in_phase}_update"
            if week_key in events:
                history += f"\nWeek {week_global}: {events[week_key]}"

        # Generate wearable updates every 2 weeks (overwrite)
        if week_global % 2 == 0:
            wearable_update = generate_wearable_update(client, week_global)
            state["last_wearable_update"] = {"week": week_global, "update": wearable_update}
            log_state_change(state, f"Updated wearable data for week {week_global}")

        # Generate member questions every week (overwrite)
        member_qs = generate_member_questions(client, week_global)
        state["last_member_questions"] = {"week": week_global, "questions": member_qs}
        log_state_change(state, f"Updated member questions for week {week_global}")

        # Generate WhatsApp-style messages
        messages = generate_week_conversation(week_global, state, history)
        weekly_conversations[f"Week_{week_global}"] = messages
        history += f"\nMessages: {messages}\n"

        # Summarize and update state (last week summary overwrite only)
        conversation_summary = summarize_conversation(messages)
        update_state(state_file_path, state, week_global, conversation_summary)
        log_state_change(state, f"Updated week {week_global} summary")

        with open(state_file_path, "w") as f:
            json.dump(state, f, indent=2)

        print(f"✅ Week_{week_global} done")

    # Store all conversations from this phase (append, not overwrite)
    all_conversations[f"Phase_{phase}"] = weekly_conversations
    with open(conversations_file_path, "w") as f:
        json.dump(all_conversations, f, indent=2)

    print(f"=== PHASE {phase} COMPLETE ===")

print(f"\nAll conversations saved to {conversations_file_path}\n")
print(f"State changes logged to {state_log_path}\n")
